# 03 - Monte Carlo Pricing and SDE Discretization

Estimate option prices via Monte Carlo, report confidence intervals, and compare Euler discretization bias.

In [4]:
%cd /content
!git clone https://github.com/basarr/interactive_portfolio_optimization.git


/content
Cloning into 'interactive_portfolio_optimization'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 63 (delta 17), reused 43 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (63/63), 41.79 KiB | 10.45 MiB/s, done.
Resolving deltas: 100% (17/17), done.


In [5]:
from pathlib import Path
ROOT = Path("/content/interactive_portfolio_optimization")
print("repo exists:", ROOT.exists())
print("src exists:", (ROOT / "src").exists())
print("notebooks exists:", (ROOT / "notebooks").exists())


repo exists: True
src exists: True
notebooks exists: True


In [6]:
import sys
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))
print("ROOT set:", ROOT)


ROOT set: /content/interactive_portfolio_optimization


In [7]:
from pathlib import Path
import sys

ROOT = Path.cwd()
if not (ROOT / 'src').exists():
    ROOT = ROOT.parent
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

import pandas as pd

from src.config import config_dict
from src.black_scholes import bs_call_price
from src.monte_carlo import mc_price_european_gbm_terminal
from src.plotting import plot_mc_ci
from src.sde import error_vs_step_count_experiment

cfg = config_dict(fast_mode=True)


In [9]:
from pathlib import Path
import sys

ROOT = Path("/content/interactive_portfolio_optimization")
if not (ROOT / "src").exists():
    raise FileNotFoundError(f"Bad ROOT: {ROOT}")

for p in [
    ROOT / "results",
    ROOT / "results" / "tables",
    ROOT / "results" / "figures",
    ROOT / "results" / "logs",
    ROOT / "results" / "reports",
]:
    p.mkdir(parents=True, exist_ok=True)

if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

print("ROOT:", ROOT)
print("tables dir exists:", (ROOT / "results" / "tables").exists())

ROOT: /content/interactive_portfolio_optimization
tables dir exists: True


In [10]:
bs = bs_call_price(cfg['S0'], cfg['K'], cfg['R'], cfg['Q'], cfg['SIGMA'], cfg['T'])
path_grid = [5_000, 20_000, 50_000]
rows = []
for n in path_grid:
    out = mc_price_european_gbm_terminal(
        S0=cfg['S0'], K=cfg['K'], r=cfg['R'], q=cfg['Q'], sigma=cfg['SIGMA'], T=cfg['T'],
        n_paths=n, option_type='call', antithetic=True, seed=cfg['SEED']
    )
    rows.append({'n_paths': n, **out, 'bs_price': bs, 'bs_in_ci': out['ci_low'] <= bs <= out['ci_high']})

mc_df = pd.DataFrame(rows)
mc_df.to_csv(ROOT / 'results' / 'tables' / 'mc_vs_bs_ci.csv', index=False)
plot_mc_ci(mc_df, bs, ROOT / 'results' / 'figures' / 'mc_ci_plot.png')
mc_df


,n_paths,price,std_error,ci_low,ci_high,antithetic,runtime_seconds,bs_price,bs_in_ci
0,5000,8.912065,0.197743,8.524495,9.299634,True,0.001617,8.916037,True
1,20000,8.941028,0.098697,8.747585,9.134471,True,0.002886,8.916037,True
2,50000,8.917881,0.061909,8.796542,9.039220,True,0.003550,8.916037,True


In [11]:
anti = mc_price_european_gbm_terminal(
    S0=cfg['S0'], K=cfg['K'], r=cfg['R'], q=cfg['Q'], sigma=cfg['SIGMA'], T=cfg['T'],
    n_paths=20_000, option_type='call', antithetic=True, seed=cfg['SEED']
)
plain = mc_price_european_gbm_terminal(
    S0=cfg['S0'], K=cfg['K'], r=cfg['R'], q=cfg['Q'], sigma=cfg['SIGMA'], T=cfg['T'],
    n_paths=20_000, option_type='call', antithetic=False, seed=cfg['SEED']
)
print('Std error (antithetic):', anti['std_error'])
print('Std error (plain):', plain['std_error'])

bias_df = error_vs_step_count_experiment(
    S0=cfg['S0'], r=cfg['R'], q=cfg['Q'], sigma=cfg['SIGMA'], T=cfg['T'],
    n_paths=20_000, step_counts=[12, 52, 126, 252], seed=cfg['SEED']
)
bias_df.to_csv(ROOT / 'results' / 'tables' / 'euler_bias_table.csv', index=False)
bias_df


Std error (antithetic): 0.09869721427770403
Std error (plain): 0.09866533411149432


,n_steps,euler_mean_ST,exact_mean_ST,theoretical_mean_ST,abs_error_euler_mean,abs_error_exact_mean
0,12,102.007787,102.017016,102.020134,0.012347,0.003118
1,52,102.011740,102.014267,102.020134,0.008394,0.005867
2,126,102.129450,102.129699,102.020134,0.109316,0.109565
3,252,102.331257,102.330263,102.020134,0.311123,0.310129



# Summary
We price a European option under risk-neutral geometric Brownian motion (GBM) using Monte Carlo simulation and compare the result to the Black–Scholes closed-form solution.

Model:
$dS_t = (r - q),S_t,dt + \sigma,S_t,dW_t.$

Exact terminal distribution:
$S_T = S_0 \exp!\left((r - q - \tfrac{1}{2}\sigma^2)T + \sigma\sqrt{T},Z\right)$,
where $Z \sim \mathcal{N}(0,1)$.

Monte Carlo pricing estimator:
$V_0 \approx e^{-rT},\frac{1}{N}\sum_{i=1}^N \text{payoff}!\left(S_T^{(i)}\right).$

A $95%$ confidence interval is given by:
$\text{estimate} \pm 1.96 \times \text{standard error}.$

- What we checked:

  1.	Monte Carlo price versus the Black–Scholes closed-form price.
	2.	Whether the Black–Scholes price lies inside the Monte Carlo confidence interval.
	3.	Variance reduction using antithetic variates $(Z,,-Z)$.
	4.	Euler path discretization bias relative to the exact GBM as the number of time steps increases.

- Key takeaways:
	  
    1. Monte Carlo pricing consists of simulating many future paths and averaging discounted payoffs.
	  
    2. Black–Scholes provides the analytic benchmark under the same assumptions.
	  
    3. Increasing the number of simulation paths $N$ leads to a tighter confidence interval.
	  
    4.	Finer time discretization reduces Euler scheme bias relative to the exact GBM solution.